In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


data = pd.read_csv("AAPL.csv")  


print(data.head())


data['Date'] = pd.to_datetime(data['Date'])


data.set_index('Date', inplace=True)


scaler = StandardScaler()
data['Close'] = scaler.fit_transform(data[['Close']])


train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)




In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense

def create_lstm_model(input_shape, num_lstm_layers=1, lstm_units=50, dropout_rate=0.2):
    model = Sequential()
    
    model.add(LSTM(units=lstm_units, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    

    for _ in range(num_lstm_layers - 1):
        model.add(LSTM(units=lstm_units, return_sequences=True))
        model.add(Dropout(dropout_rate))
    
   
    model.add(LSTM(units=lstm_units))
    model.add(Dropout(dropout_rate))
    
    
    model.add(Dense(units=1))
    
    return model


input_shape = (10, 1)  


model = create_lstm_model(input_shape, num_lstm_layers=2, lstm_units=64, dropout_rate=0.2)

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')


X_train = train_data.drop(columns=['Consumption']).values
y_train = train_data['Consumption'].values
X_test = test_data.drop(columns=['Consumption']).values
y_test = test_data['Consumption'].values


X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))


history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, verbose=1)